In [ ]:
import os
import cv2
import torch

# Config — update these paths as needed
yolov5_repo = r"D:/repos/intro_to_ml/yolov5"
weights = r"D:/repos/intro_to_ml/runs/train/exp_yolov5l_bs32_ep150/weights/best.pt"
img_path = r"D:/repos/intro_to_ml/data/rdd2022/sample/China_MotorBike_000778.jpg"
out_dir = r"D:/repos/intro_to_ml/runs/detect/single_test_2025_10_09/China_MotorBike_000329.jpg"
save_conf = True          # set False to omit confidence column in .txt
label_format = "yolo"     # "yolo" for normalized xc yc w h, "xyxy" for absolute x1 y1 x2 y2

os.makedirs(out_dir, exist_ok=True)
labels_out = os.path.join(out_dir, "labels")
os.makedirs(labels_out, exist_ok=True)

# Load model (local yolov5)
model = torch.hub.load(yolov5_repo, 'custom', path=weights, source='local')
model.conf = 0.25
model.iou = 0.45

# Read image (BGR)
img_bgr = cv2.imread(img_path)
if img_bgr is None:
    raise FileNotFoundError(f"Image not found: {img_path}")
h, w = img_bgr.shape[:2]

# Inference (model expects RGB)
img_rgb = img_bgr[:, :, ::-1]
results = model(img_rgb, size=640)

# Get predictions
df = results.pandas().xyxy[0]  # xmin, ymin, xmax, ymax, confidence, class, name
print(df)  # inspect if you want

# Draw boxes on image (same as before)
for _, row in df.iterrows():
    x1 = int(max(0, row['xmin'])); y1 = int(max(0, row['ymin']))
    x2 = int(min(w-1, row['xmax'])); y2 = int(min(h-1, row['ymax']))
    conf = float(row['confidence']); cls_name = str(row['name'])
    label = f"{cls_name} {conf:.2f}"
    cv2.rectangle(img_bgr, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(img_bgr, label, (x1, max(15, y1-6)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1)

# Save annotated image
out_image_path = os.path.join(out_dir, os.path.basename(img_path))
ok = cv2.imwrite(out_image_path, img_bgr)
print("Saved image:", out_image_path, "ok=", ok)

# Save prediction .txt in YOLO format (or XYXY)
base = os.path.splitext(os.path.basename(img_path))[0]
out_txt = os.path.join(labels_out, base + ".txt")

lines = []
if len(df) == 0:
    # Create empty .txt (YOLOv5 expects an empty file if no objects)
    open(out_txt, "w").close()
    print("No detections — wrote empty file:", out_txt)
else:
    for _, row in df.iterrows():
        cls_id = int(row['class'])  # numeric class id
        if label_format == "yolo":
            # normalized center x,y and w,h in [0,1]
            x_center = ((row['xmin'] + row['xmax']) / 2.0) / w
            y_center = ((row['ymin'] + row['ymax']) / 2.0) / h
            bw = (row['xmax'] - row['xmin']) / w
            bh = (row['ymax'] - row['ymin']) / h
            if save_conf:
                lines.append(f"{cls_id} {x_center:.6f} {y_center:.6f} {bw:.6f} {bh:.6f} {row['confidence']:.6f}")
            else:
                lines.append(f"{cls_id} {x_center:.6f} {y_center:.6f} {bw:.6f} {bh:.6f}")
        else:  # "xyxy"
            if save_conf:
                lines.append(f"{cls_id} {int(row['xmin'])} {int(row['ymin'])} {int(row['xmax'])} {int(row['ymax'])} {row['confidence']:.6f}")
            else:
                lines.append(f"{cls_id} {int(row['xmin'])} {int(row['ymin'])} {int(row['xmax'])} {int(row['ymax'])}")

    with open(out_txt, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))
    print("Wrote labels:", out_txt)

YOLOv5  v7.0-430-g459d8bf0 Python-3.13.3 torch-2.8.0+cu126 CUDA:0 (NVIDIA RTX A6000, 49140MiB)

Fusing layers... 
Model summary: 267 layers, 46129818 parameters, 0 gradients, 107.7 GFLOPs
Adding AutoShape... 


   xmin        ymin        xmax        ymax  confidence  class       name
0   0.0  173.876068  493.524506  511.756744    0.798432      2  Crocodile
Saved image: D:/repos/intro_to_ml/runs/detect/single_2025_10_09/China_MotorBike_000329.jpg\China_MotorBike_000778.jpg ok= True
Wrote labels: D:/repos/intro_to_ml/runs/detect/single_2025_10_09/China_MotorBike_000329.jpg\labels\China_MotorBike_000778.txt


D:\repos/intro_to_ml/yolov5\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
